In [2]:
from PIL import Image
import os

input_base_folder = 'Dataset Proper/Test'
output_base_folder = 'New Dataset_proper_resized/test'

# List of class folders
class_folders = os.listdir(input_base_folder)

# Iterate through class folders
for class_folder in class_folders:
    input_folder = os.path.join(input_base_folder, class_folder)
    output_folder = os.path.join(output_base_folder, class_folder)

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Resize images and save them to the output folder
    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)

        # Load the image using PIL
        image = Image.open(image_path)

        # Resize the image to your desired size (e.g., 224x224)
        target_size = (224, 224)
        image = image.resize(target_size)

        # Save the resized image to the output folder
        save_path = os.path.join(output_folder, filename)
        image.save(save_path)

print("Resized images done")


Resized images done


In [4]:
import tensorflow as tf
import cv2
import os

# Path to the directory containing the Herlev dataset
dataset_dir = 'New Dataset_proper_resized/test'

# Detect classes automatically
classes = [class_name for class_name in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, class_name))]

# Define the NLM filter function
def nlm_filter(image):
    filtered_image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)  # Adjust parameters as needed
    return filtered_image

# Iterate through each class
for class_name in classes:
    class_dir = os.path.join(dataset_dir, class_name)
    output_dir = os.path.join('New NLM Filter/test', class_name, f'{class_name}_filtered_nlm')
    os.makedirs(output_dir, exist_ok=True)

    # List all image files in the class directory
    image_files = [os.path.join(class_dir, filename) for filename in os.listdir(class_dir) if filename.lower().endswith('.bmp')]

    # Apply NLM filtering to each image, convert to PNG, and save the filtered images
    for image_file in image_files:
        # Load the image using OpenCV
        image = cv2.imread(image_file)

        # Convert the image to RGB (OpenCV reads images in BGR format)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Apply NLM filtering
        filtered_image = nlm_filter(image_rgb)

        # Get the file name without extension
        filename = os.path.splitext(os.path.basename(image_file))[0]

        # Save the filtered image as PNG format
        output_file = os.path.join(output_dir, f'{filename}_filtered.png')
        cv2.imwrite(output_file, filtered_image)

print("Noise removal and PNG conversion completed.")


Noise removal and PNG conversion completed.
